In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from smooth.adam_general.core.adam import ADAM
import numpy as np
import pandas as pd

# Simulation-Based Prediction Intervals - Python Implementation

This notebook tests simulation-based prediction intervals in Python to compare with R.

Key points:
- Read the same test data generated by R
- Use `interval_method='simulation'` in predict()
- Compare lower/upper bounds with R results

**Note**: For 100% identical results, both R and Python must use:
- Same input data (from CSV)
- Same model parameters (may need to fix parameters to match)
- Same random seed for simulation

In [3]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

### Test 1: Simple ETS(A,N,N) with simulation intervals

In [4]:
# Load test data from R
print(f'Loaded {len(ts_df)} data points')
print(ts_df.head())

Loaded 100 data points
                value
2023-01-31  96.811465
2023-02-28  83.970194
2023-03-31  84.647821
2023-04-30  94.295991
2023-05-31  97.832717


In [5]:
# Fit model
model = ADAM(model='ANN', lags=[12], distribution='dnorm',initial = 'optimal')
model.fit(ts_df)

In [6]:
# Forecast with simulation intervals
# Fit model
model = ADAM(model='ANN', lags=[12], distribution='dnorm',initial = 'optimal')
model.fit(ts_df)
result = model.predict(h=12, interval_method='simulation', nsim=1000, level=0.95)

print('\nSimulation-based intervals (nsim=1000):')
print(model.forecast_results)


Simulation-based intervals (nsim=1000):
                  mean  lower_0.025  upper_0.975
2031-05-31  101.133962    81.278420   122.349657
2031-06-30  101.133962    80.816060   121.873239
2031-07-31  101.133962    81.220706   123.620008
2031-08-31  101.133962    78.727017   121.713111
2031-09-30  101.133962    79.405041   122.513029
2031-10-31  101.133962    78.866961   121.949684
2031-11-30  101.133962    79.214961   119.903742
2031-12-31  101.133962    80.753263   120.685839
2032-01-31  101.133962    80.705458   121.077135
2032-02-29  101.133962    81.453599   121.811136
2032-03-31  101.133962    80.007028   121.213533
2032-04-30  101.133962    80.521393   121.927289


In [7]:
# Display intervals more clearly
print('\nForecast mean:')
print(model.forecast_results['mean'].values)
print('\nLower bound (2.5%):')
print(model.forecast_results.filter(like='lower').values.flatten())
print('\nUpper bound (97.5%):')
print(model.forecast_results.filter(like='upper').values.flatten())


Forecast mean:
[101.13396235 101.13396235 101.13396235 101.13396235 101.13396235
 101.13396235 101.13396235 101.13396235 101.13396235 101.13396235
 101.13396235 101.13396235]

Lower bound (2.5%):
[81.27842044 80.81606027 81.22070641 78.72701686 79.40504057 78.86696056
 79.21496058 80.75326266 80.70545793 81.45359892 80.0070284  80.52139267]

Upper bound (97.5%):
[122.34965696 121.87323857 123.62000831 121.71311072 122.51302863
 121.94968397 119.90374248 120.68583863 121.07713526 121.81113589
 121.21353331 121.92728926]


### Test 2: ETS(A,A,N) with trend

In [8]:
# Forecast with simulation intervals
np.random.seed(123)
result = model.predict(h=12, interval_method='simulation', nsim=1000, level=0.95)

print('\nSimulation-based intervals:')
print('Mean:', model.forecast_results['mean'].values[:3])
print('Lower:', model.forecast_results.filter(like='lower').values.flatten()[:3])
print('Upper:', model.forecast_results.filter(like='upper').values.flatten()[:3])


Simulation-based intervals:
Mean: [101.13396235 101.13396235 101.13396235]
Lower: [81.12481036 81.50941152 78.54720457]
Upper: [120.60598496 122.51011835 121.16154234]


### Test 3: Seasonal ETS(A,N,A)

In [9]:
# Fit model
model = ADAM(model='ANA', lags=[12], distribution='dnorm', frequency='ME')
model.fit(ts_df)

print('ETS(A,N,A) Model fitted')
print('Alpha:', model.adam_estimated['B'][0])
print('Gamma:', model.adam_estimated['B'][1])

ETS(A,N,A) Model fitted
Alpha: 0.05539950418213001
Gamma: 0.001123442536020658


In [10]:
# Forecast with simulation intervals
np.random.seed(123)
result = model.predict(h=12, interval_method='simulation', nsim=1000, level=0.95)

print('\nSimulation-based intervals:')
print('Mean:', model.forecast_results['mean'].values[:3])
print('Lower:', model.forecast_results.filter(like='lower').values.flatten()[:3])
print('Upper:', model.forecast_results.filter(like='upper').values.flatten()[:3])


Simulation-based intervals:
Mean: [102.92650143 109.37646958  97.44564223]
Lower: [80.64282742 87.81669107 74.34447687]
Upper: [126.2074318  131.81512756 119.4858946 ]


### Test 4: Multiplicative ETS(M,A,M)

In [11]:
# Forecast with simulation intervals
np.random.seed(123)
result = model.predict(h=12, interval_method='simulation', nsim=1000, level=0.95)

print('\nSimulation-based intervals:')
print('Mean:', model.forecast_results['mean'].values[:3])
print('Lower:', model.forecast_results.filter(like='lower').values.flatten()[:3])
print('Upper:', model.forecast_results.filter(like='upper').values.flatten()[:3])


Simulation-based intervals:
Mean: [102.92650143 109.37646958  97.44564223]
Lower: [80.24962161 87.22350775 75.88785543]
Upper: [125.19872845 132.37049329 121.25737635]


### Test 5: Compare parametric vs simulation intervals

In [12]:
# Use the ANN data

model = ADAM(model='ANN', lags=[1], distribution='dnorm', frequency='ME')
model.fit(ts_df)

In [13]:
# Parametric intervals
result_param = model.predict(h=12, interval_method='parametric', level=0.95)
print('Parametric intervals:')
print('Lower:', model.forecast_results.filter(like='lower').values.flatten()[:3])
print('Upper:', model.forecast_results.filter(like='upper').values.flatten()[:3])

Parametric intervals:
Lower: [80.55160478 80.54516092 80.53871908]
Upper: [121.71631992 121.72276378 121.72920562]


In [14]:
# Simulation intervals
np.random.seed(123)
result_sim = model.predict(h=12, interval_method='simulation', nsim=10000, level=0.95)
print('\nSimulation intervals (nsim=10000):')
print('Lower:', model.forecast_results.filter(like='lower').values.flatten()[:3])
print('Upper:', model.forecast_results.filter(like='upper').values.flatten()[:3])


Simulation intervals (nsim=10000):
Lower: [80.39619247 80.26405636 80.23402362]
Upper: [121.98566528 121.5827444  121.87807512]


### Test 6: Different nsim values

In [15]:
# Test with different nsim values
model = ADAM(model='ANN', lags=[1], distribution='dnorm', frequency='ME')
model.fit(ts_df)

for nsim_val in [100, 500, 1000, 5000]:
    np.random.seed(123)
    model.predict(h=5, interval_method='simulation', nsim=nsim_val, level=0.95)
    lower = model.forecast_results.filter(like='lower').values.flatten()[0]
    upper = model.forecast_results.filter(like='upper').values.flatten()[0]
    print(f'nsim={nsim_val}: Lower={lower:.2f}, Upper={upper:.2f}')

nsim=100: Lower=78.66, Upper=120.77
nsim=500: Lower=80.18, Upper=122.62
nsim=1000: Lower=80.54, Upper=122.11
nsim=5000: Lower=79.76, Upper=122.43


## Summary

Python simulation intervals work by:
1. Generating random errors from the fitted distribution
2. Running multiple simulation paths through the state-space model (via C++ adam_simulator)
3. Taking quantiles of the simulated values

Key parameters:
- `interval_method='simulation'` in predict()
- `nsim` controls number of simulations (default 10000)
- `level` sets the confidence level

### Comparison Notes

To get 100% identical results with R:
1. Use the same input data (CSV files)
2. Ensure model parameters match (may need to fix some parameters)
3. Random number generation differs between R and Python - exact matches require using the same error matrix